In [ ]:
from tqdm import tqdm
from itertools import combinations
from pathlib import Path
import sys
import warnings
import logging
from pprint import pformat
import traceback
import os

import numpy as np
import pandas as pd
from pmdarima.arima import ARIMA, auto_arima
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, 
from tensorflow.keras.regularizers import l1_l2
import dynamic_yaml
import yaml

sys.path.append("/tf/correlation-coef-predict/ywt_library")
import data_generation
from data_generation import data_gen_cfg
from ywt_arima import arima_model, arima_err_logger_init

with open('../config/data_config.yaml') as f:
    data = dynamic_yaml.load(f)
    data_cfg = yaml.full_load(dynamic_yaml.dump(data))

warnings.simplefilter("ignore")
logging.basicConfig(level=logging.INFO)


# %load_ext pycodestyle_magic
# %pycodestyle_on --ignore E501
logging.info(pformat(data_cfg, indent=1, width=100, compact=True))

# Prepare data

## Data implement & output setting & trainset setting

In [ ]:
model_dir = Path('./save_models/')
lstm_log_dir = Path('./save_models/lstm_train_logs/')
res_dir = Path('./results/')
model_dir.mkdir(parents=True, exist_ok=True)
lstm_log_dir.mkdir(parents=True, exist_ok=True)
res_dir.mkdir(parents=True, exist_ok=True)

# setting of output files
save_corr_data = True
save_arima_resid_data = True
# data implement setting
data_implement = "SP500_20082017_FREQ_CLUSTER_TEST"  # watch options by operate: print(data_cfg["DATASETS"].keys())
# train set setting
train_items_setting = "-train_train"  # -train_train|-train_all
# data split  period setting, only suit for only settings of Korean paper
data_split_settings = ["-data_sp_train", "-data_sp_dev", "-data_sp_test1", "-data_sp_test2", ]
# lstm_hyper_params
lstm_hyper_param = "-kS_hyper"

In [ ]:
# data loading & implement setting
dataset_df = pd.read_csv(data_cfg["DATASETS"][data_implement]['FILE_PATH'])
dataset_df = dataset_df.set_index('Date')
all_set = list(dataset_df.columns)  # all data
train_set = data_cfg["DATASETS"][data_implement]['TRAIN_SET']
test_set = data_cfg['DATASETS'][data_implement]['TEST_SET'] if data_cfg['DATASETS'][data_implement].get('TEST_SET') else [p for p in all_set if p not in train_set]  # all data - train data
logging.info(f"===== len(train_set): {len(train_set)}, len(all_set): {len(all_set)}, len(test_set): {len(test_set)} =====")

# train items implement settings
items_implement = train_set if train_items_setting == "-train_train" else all_set
logging.info(f"===== len(train set): {len(items_implement)} =====")

# setting of name of output files and pictures title
output_file_name = data_cfg["DATASETS"][data_implement]['OUTPUT_FILE_NAME_BASIS'] + train_items_setting
logging.info(f"===== file_name basis:{output_file_name} =====")

# display(dataset_df)

## Load or Create Correlation Data

In [ ]:
corr_data_dir = Path(data_cfg["DIRS"]["PIPELINE_DATA_DIR"])/f"{output_file_name}-corr_data"
corr_data_dir.mkdir(parents=True, exist_ok=True)
data_length = int(len(dataset_df)/data_gen_cfg["CORR_WINDOW"])*data_gen_cfg["CORR_WINDOW"]
corr_ser_len_max = int((data_length-data_gen_cfg["CORR_WINDOW"])/data_gen_cfg["CORR_STRIDE"])

train_df_path = corr_data_dir/f"{output_file_name}-corr_train.csv"
dev_df_path = corr_data_dir/f"{output_file_name}-corr_dev.csv"
test1_df_path = corr_data_dir/f"{output_file_name}-corr_test1.csv"
test2_df_path = corr_data_dir/f"{output_file_name}-corr_test2.csv"
all_corr_df_paths = dict(zip(["train_df", "dev_df", "test1_df", "test2_df"],
                             [train_df_path, dev_df_path, test1_df_path, test2_df_path]))
if all([df_path.exists() for df_path in all_corr_df_paths.values()]):
    corr_datasets = [pd.read_csv(df_path).set_index("items") for df_path in all_corr_df_paths.values()]
else:
    corr_datasets = data_generation.gen_train_data(items_implement, raw_data_df=dataset_df, corr_df_paths=all_corr_df_paths, corr_ser_len_max=corr_ser_len_max, save_file=save_corr_data)

# LSTM model for first stage prediciton

In [ ]:
model_log = keras.callbacks.TensorBoard(lstm_log_dir=lstm_log_dir)
model_earlystop = tf.keras.callbacks.EarlyStopping(patience=50, monitor="val_loss")
save_model = keras.callbacks.ModelCheckpoint(Path(model_dir)/"epoch_{epoch}_{val_loss:.5f}.h5",
                                             monitor='val_loss', verbose=1, mode='min', save_best_only=True)
callbacks_list = [model_log, model_earlystop, save_model]
inputs = Input(shape=(20, 1))
lstm_1 = LSTM(units=20, kernel_regularizer=l1_l2(0.0, 0.0), bias_regularizer=l1_l2(0.0, 0.0))(inputs)
outputs = Dense(units=21, activation="relu")(lstm_1)
first_stage_lstm_model = keras.Model(inputs, outputs, name="first_stage_lstm")
first_stage_lstm_model.summary()
first_stage_lstm_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae'])
train_history = first_stage_lstm_model.fit(x=corr_datasets[0], y=corr_datasets[0], validation_data=(corr_datasets[1], corr_datasets[1]), epochs=500, batch_size=64, shuffle=True, callbacks=callbacks_list, verbose=1)

In [ ]:
best_epoch_num = np.argmin(np.array(train_history.history['val_loss'])) + 1
best_val_loss = train_history.history['val_loss'][epoch_num-1]
best_first_stage_lstm_model = load_model(Path(model_dir)/f"epoch_{best_epoch_num}_{best_val_loss:.5f}.h5")

In [ ]:
for (file_name, dataset) in tqdm(zip(['train', 'dev', 'test1', 'test2'], corr_datasets)):
    (dataset, save_file_name=file_name)

# LSTM for residual

In [ ]:
# Dataset.from_tensor_slices(dict(pd.read_csv(f'./dataset/after_arima/arima_resid_train.csv')))
lstm_train_X = pd.read_csv(f'./dataset/after_arima/{output_file_name}_arima_resid_train.csv').set_index('Unnamed: 0').iloc[::, :-1]
lstm_train_Y = pd.read_csv(f'./dataset/after_arima/{output_file_name}_arima_resid_train.csv').set_index('Unnamed: 0').iloc[::, -1]
lstm_dev_X = pd.read_csv(f'./dataset/after_arima/{output_file_name}_arima_resid_dev.csv').set_index('Unnamed: 0').iloc[::, :-1]
lstm_dev_Y = pd.read_csv(f'./dataset/after_arima/{output_file_name}_arima_resid_dev.csv').set_index('Unnamed: 0').iloc[::, -1]
lstm_test1_X = pd.read_csv(f'./dataset/after_arima/{output_file_name}_arima_resid_test1.csv').set_index('Unnamed: 0').iloc[::, :-1]
lstm_test1_Y = pd.read_csv(f'./dataset/after_arima/{output_file_name}_arima_resid_test1.csv').set_index('Unnamed: 0').iloc[::, -1]
lstm_test2_X = pd.read_csv(f'./dataset/after_arima/{output_file_name}_arima_resid_test2.csv').set_index('Unnamed: 0').iloc[::, :-1]
lstm_test2_Y = pd.read_csv(f'./dataset/after_arima/{output_file_name}_arima_resid_test2.csv').set_index('Unnamed: 0').iloc[::, -1]
num_samples = len(lstm_train_X)

lstm_train_X = lstm_train_X.values.reshape(num_samples, 20, 1)
lstm_train_Y = lstm_train_Y.values.reshape(num_samples, 1)
lstm_dev_X = lstm_dev_X.values.reshape(num_samples, 20, 1)
lstm_dev_Y = lstm_dev_Y.values.reshape(num_samples, 1)
lstm_test1_X = lstm_test1_X.values.reshape(num_samples, 20, 1)
lstm_test1_Y = lstm_test1_Y.values.reshape(num_samples, 1)
lstm_test2_X = lstm_test2_X.values.reshape(num_samples, 20, 1)
lstm_test2_Y = lstm_test2_Y.values.reshape(num_samples, 1)

In [ ]:
def double_tanh(x):
    return (tf.math.tanh(x) *2)


def build_many_one_lstm():
    inputs = Input(shape=(20, 1))
    lstm_1 = LSTM(units=25, kernel_regularizer=l1_l2(0.0, 0.0), bias_regularizer=l1_l2(0.0, 0.0))(inputs)
    outputs = Dense(units=1, activation=double_tanh)(lstm_1)
    return keras.Model(inputs, outputs, name="many_one_lstm")


lstm_model = build_many_one_lstm()
lstm_model.summary()
lstm_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae'])

In [ ]:
model_dir = './models'
log_dir = './models/lstm_train_logs'
res_dir = './results'
os.makedirs(res_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
res_csv_path = Path(res_dir+f'/{output_file_name}_LSTM_evaluation.csv')
res_csv_path.touch(exist_ok=True)
with open(res_csv_path, 'r+') as f:
    if not f.read():
        f.write("epoch,TRAIN_MSE,DEV_MSE,TEST1_MSE,TEST2_MSE,TRAIN_MAE,DEV_MAE,TEST1_MAE,TEST2_MAE")

res_df = pd.read_csv(res_csv_path)
saved_model_list = [int(p.stem.split('_')[1]) for p in Path(model_dir).glob('*.h5')]
model_log = keras.callbacks.TensorBoard(log_dir=log_dir)
epoch_start = max(saved_model_list) if saved_model_list else 1
max_epoch = 3000

for epoch_num in range(epoch_start, max_epoch):
    if epoch_num > 1:
        lstm_model = load_model(Path(model_dir)/f"{output_file_name}_epoch_{epoch_num - 1}.h5", custom_objects={'double_tanh':double_tanh})

    save_model = keras.callbacks.ModelCheckpoint(Path(model_dir)/f"{output_file_name}_epoch_{epoch_num}.h5",
                                                 monitor='loss', verbose=1, mode='min', save_best_only=False)
    lstm_model.fit(lstm_train_X, lstm_train_Y, epochs=1, batch_size=500, shuffle=True, callbacks=[model_log, save_model])
    
    # test the model
    score_train = lstm_model.evaluate(lstm_train_X, lstm_train_Y)
    score_dev = lstm_model.evaluate(lstm_dev_X, lstm_dev_Y)
    score_test1 = lstm_model.evaluate(lstm_test1_X, lstm_test1_Y)
    score_test2 = lstm_model.evaluate(lstm_test2_X, lstm_test2_Y)
    res_each_epoch_df = pd.DataFrame(np.array([epoch_num, score_train[0], score_dev[0], 
                                               score_test1[0], score_test2[0], 
                                               score_train[1], score_dev[1], 
                                               score_test1[1], score_test2[1]]).reshape(-1, 9),
                                    columns=["epoch", "TRAIN_MSE", "DEV_MSE", "TEST1_MSE", 
                                             "TEST2_MSE", "TRAIN_MAE", "DEV_MAE",
                                             "TEST1_MAE","TEST2_MAE"])
    res_df = pd.concat([res_df, res_each_epoch_df])

res_df.to_csv(res_csv_path, index=False)